In [4]:
from src import svm as svm
from src import util as util
import numpy as np
import matplotlib.pyplot as plt
import collections
import csv

ModuleNotFoundError: No module named 'src'

In [ ]:
def load_spam_dataset_csv(csv_path):
    """Load the spam dataset from a TSV file

    Args:
         csv_path: Path to TSV file containing dataset.

    Returns:
        messages: A list of string values containing the text of each message.
        labels: The binary labels (0 or 1) for each message. A 1 indicates spam.
    """

    messages = []
    labels = []

    with open(csv_path, 'r', newline='', encoding='utf8') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')

        for message, label in reader:
            messages.append(message)
            labels.append(1 if label == '1' else 0)

    return messages, np.array(labels)

In [2]:
train_messages, train_labels = util.load_spam_dataset('data/ds6_train.tsv')
test_messages, test_labels = util.load_spam_dataset('data/ds6_test.tsv')
train2_messages, train2_labels = load_spam_dataset_csv('data/emails.csv')
val_messages, val_labels = util.load_spam_dataset('data/ds6_val.tsv')

NameError: name 'util' is not defined

In [3]:
def getwords(message):
    return message.lower().split()

In [11]:
def h(theta,x):
    return 1 / (1 + np.exp(-np.dot(x, theta)))

In [12]:
def Create_dictionary(messages):
    words = [word for message in messages for word in getwords(message)]
    words_count = collections.Counter(words)
    #word_dic = {(word, count) for word,count in words_count.items() if count>=5}
    freq_word = [word for word,count in words_count.items() if count>=10 and word !='subject:']
    return {word: count for count, word in enumerate(freq_word)}

In [13]:
dicWords = Create_dictionary(train_messages)

In [59]:
vocabulary = Create_dictionary(train2_messages)

In [60]:
def Transform_text(messages, word_dictionary):
    m = len(messages)
    n = len(word_dictionary)
    matrix = np.zeros((m,n), dtype=int)

    word_counts = [collections.Counter(getwords(message)) for message in messages]
    for i in range(m):
        for word, count in word_counts[i].items():
            if word in word_dictionary:
                matrix[i][word_dictionary[word]] += count

    return matrix


In [15]:
train_matrix = Transform_text(train_messages,dicWords)

In [62]:
train2_matrix = Transform_text(train2_messages,vocabulary)

In [77]:
test_matrix = Transform_text(test_messages,vocabulary)

In [78]:
X_with_bias = np.c_[np.ones(train_matrix.shape[0]),train_matrix]

In [79]:
X2_with_bias = np.c_[np.ones(train2_matrix.shape[0]),train2_matrix]

In [80]:
X_test_with_bias = np.c_[np.ones(test_matrix.shape[0]),test_matrix]

In [81]:
X_test_with_bias.shape

(558, 8070)

In [82]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X2_with_bias,train2_labels)

LogisticRegression(max_iter=1000)

In [110]:

LR_predictions = log_reg.predict(X_test_with_bias)


In [111]:
LR_accuracy = np.mean(LR_predictions == test_labels)

In [112]:
print('Logistic Regression with large dataset had an accuracy of {} on the testing set'.format(LR_accuracy))

Logistic Regression with large dataset had an accuracy of 0.7132616487455197 on the testing set


In [135]:
simpletesting=[
'Get new phone free Click this URL',
"Don't forget to Subscribe our page in www.hh.com",
"Urgent your Account has been compromised. Verify your details now",
    "Making $5000 a week working from home. No Skills required",
"Your Package delivery failed. Click here to reschedule",
    "Act now this offer expires in 24 hours",
    "Get rich quick with this one simple trick",
    "Warning "
]

In [136]:
testing1_matrix = Transform_text(simpletesting,vocabulary)
testing1_matrix_with_bias = np.c_[np.ones(testing1_matrix.shape[0]),testing1_matrix]

In [137]:
LR_predictions2 = log_reg.predict(testing1_matrix_with_bias)
LR_predictions2

array([1, 0, 1, 1, 1, 1, 1])